In [67]:
# for grabbing/handling data
import numpy as np
import pandas as pd
import fukushima_telomere_methods as f_tm

# incase needing to reload modules
import importlib
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


---
&nbsp; 

The Fukushima Project seeks to determine whether regions contaminated by radiation from the Fukushima Daiichi Nuclear Disaster can be considered safe for human repopulation. Here, we use free-roaming >200 wild boar trapped from the contaminated regions as mammalian proxies for humans. On the boar we've collected >10 types of data from chemistry, physics (radiation), and molecular biology techniques to ascertain whether boar are negatively impacted by radiation in terms of various biometrics. We have data on snake as well.

The data was generated over >4 years and exists as tabular data in various formats. Our first aim is to read in all of the data, then clean/reformat it to enable combination of the various datasets. We've focused on standardizing the format for boar sample identifications (Sample ID) to enable combination of the data.

&nbsp; 

---

# Aryn's Data (qPCR)
---

## Reading Aryn's snake data to dataframe

In [24]:
snake_df = pd.read_excel('../excel data/raw/snake dose data-15Nov19 (2).xlsx', skiprows=4)
# snake_df.drop([20, 21, 22, 23], axis=0, inplace=True)
snake_df.rename({'Telomere Length (qPCR)':'Mean Telomere Length (qPCR)'}, axis=1, inplace=True)
snake_df.head(4)

,Sample ID,DNA Concentration (ng/mcL),260/280,Sex,External Dose Rate (uSv/h),Mean Telomere Length (qPCR),Int + Ext dose rate (uGy/h),134Cs,137Cs,134/137,134Cs int dose rate,137Cs int dose rate,Total int dose rate,134Cs ext dose rate,137Cs ext dose rate,Total ext dose rate,Total dose rates μGy h-1
0,CEC10,19.3,1.96,F,0.10,0.444298,0.30,56.0,509.0,0.110020,0.01,0.08,0.09,0.05,0.17,0.22,0.30
1,CEC18,63.7,1.83,F,0.16,0.82619,0.11,17.0,185.0,0.091892,0.00,0.03,0.03,0.02,0.06,0.08,0.11
2,CEC22,45.5,1.88,F,0.17,2.06721,0.03,6.0,55.0,0.109091,0.00,0.01,0.01,0.01,0.02,0.02,0.03
3,CEC23,32.6,1.83,F,0.20,1.38918,0.18,34.0,304.0,0.111842,0.00,0.05,0.05,0.03,0.10,0.13,0.18


In [25]:
snake_dose = pd.read_excel('../excel data/raw/Gerke_SupplementaryTable1_revised (1).xlsx', skiprows=3)
snake_dose
snake_dose = snake_dose[['Unnamed: 1', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 21', 'Unnamed: 25']].copy()
snake_dose.rename({'Unnamed: 1': 'Sample ID',
                   'Unnamed: 6': 'Total length (cm)',
                   'Unnamed: 7': 'Mass (g)',
                   'Unnamed: 21': 'Total dose rates (μGy h-1) meth1', 
                   'Unnamed: 25': 'Total dose rates (μGy h-1) meth2'},
                   axis=1, inplace=True)

In [26]:
snake_clean = snake_df[['Sample ID', 'Sex', 'Mean Telomere Length (qPCR)']].merge(snake_dose, on='Sample ID')
snake_clean.drop(list(range(20, 39)), inplace=True)
snake_clean.head(4)

,Sample ID,Sex,Mean Telomere Length (qPCR),Total length (cm),Mass (g),Total dose rates (μGy h-1) meth1,Total dose rates (μGy h-1) meth2
0,CEC10,F,0.444298,113.7,218.72,0.171601,0.304258
1,CEC18,F,0.82619,108.3,206.50,0.168549,0.10703
2,CEC22,F,2.06721,91.9,104.50,0.156359,0.0326416
3,CEC23,F,1.38918,106.7,204.50,0.22353,0.18235


In [27]:
# import re

# def natural_key(string_):
#     return [int(s) if s.isdigit() else s for s in re.split(r'(\d+)', string_) if s]

# list_snake_ID = list(snake_df['Sample ID'].unique())
# sorted(list_snake_ID, key=natural_key)

# # making column for radiation exposure status and encoded dummy variables from the new column
# aryn_snake_df = f_tm.readable_snake_df_dummy_variables(aryn_snake_df)

## Saving Aryn's Snake dataframe for later retrieval

In [28]:
snake_clean.to_csv('../excel data/cleaned/aryn_snake_df.csv', index=False)

## Reading Aryn's Boar data to dataframe

In [29]:
aryn_boar_dose = pd.read_excel('../excel data/raw/Kelly& Aryn boar_1Dec2018-edited-TH-17Nov19 (2).xlsx',
                               sheet_name=0, skiprows=71)

aryn_boar_dose_raw = aryn_boar_dose.copy()
aryn_boar_dose.drop([46, 47], axis=0, inplace=True)

aryn_boar_dose['Age (months)'] = aryn_boar_dose['hours'].apply(lambda x: int(x / (24 * 30)))
aryn_boar_dose['Sex'] = aryn_boar_dose['Sex'].apply(lambda row: f_tm.male_or_female(row))

# dropping empty columns & rows, unneccessary cols
aryn_boar_dose.drop(['Unnamed: 0', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 7',
                     'Unnamed: 9', 'Unnamed: 11', 'Unnamed: 38', 'Cs-134 Bq/kg (dry)',
                     'Cs-137 Bq/kg (dry)', 'EXTERNAL Dose Maximum Life Time, mGy (134+137)',
                     'Maximum Total Life Time Dose mGy (Int+Ext,134+137)',
                     'hours', 'Age', 'tissue', 'dry wt.'], 
                     axis=1, inplace=True)

In [30]:
aryn_boar_qPCR = pd.read_excel('../excel data/raw/boar snake master template.xlsx', sheet_name=0)
aryn_boar_qPCR.head(4)

,Exposure Status,Sample ID,Sex,Age,Age Class,Age (months),External Dose (uSv),Internal Dose (uSv),qPCR ID,Concentration (ng/mcL),260/280,Normalized T/A Average (Average Telomere Length),Cortisol,Dicentrics (TC-FISH),PH anomalies
0,Control,180526C3,M,NaN,Yearling,15,0.38,NaN,C1,14.5,1.90,0.910559,NaN,NaN,NaN
1,Control,180528C2,F,NaN,Juv,8,-,NaN,C2,17.3,1.73,0.985085,NaN,NaN,NaN
2,Control,180604C2,F,NaN,Yearling,18,0.46,NaN,C3,15.8,1.90,1.117980,NaN,NaN,NaN
3,Control,180612C1,M,NaN,Yearling,15,0.27,NaN,C4,13.1,1.60,1.260040,NaN,NaN,NaN


## Cleaning Aryn's boar dose/qPCR dataframes

In [31]:
# changing errant values to np.NaN
aryn_boar_qPCR = aryn_boar_qPCR.replace('-', np.NaN)

# cleaning Sample ID of dashes & spaces, + 'GPS' & 'collar' terms
aryn_boar_qPCR['Sample ID'] = aryn_boar_qPCR['Sample ID'].apply(lambda row: f_tm.remove_dashes_space_sampleIDs(row))
aryn_boar_qPCR['Sample ID'] = aryn_boar_qPCR['Sample ID'].astype('str')

aryn_boar_dose['Sample ID'] = aryn_boar_dose['Sample ID'].apply(lambda row: f_tm.remove_dashes_space_sampleIDs(row))
aryn_boar_dose['Sample ID'] = aryn_boar_dose['Sample ID'].astype('str')

## Saving Aryn's boar dose/qPCR data for later retrieval

In [32]:
aryn_boar_dose.to_csv('../excel data/cleaned/aryn_boar_dose.csv', index=False)
aryn_boar_qPCR.to_csv('../excel data/cleaned/aryn_boar_qPCR.csv', index=False)

# Cortisol Data
---

## Reading the boar cortisol data

In [68]:
boar_cortisol_df = pd.read_excel('../excel data/raw/boar cortisol.xlsx')
boar_cortisol_df_raw = boar_cortisol_df.copy()
boar_cortisol_df.drop([47], axis=0, inplace=True)
boar_cortisol_df.head(4)

,Biel #,Date Received,Sample Number (electronic sheet),Sample ID,Hair wt (mg),Cortisol (pg/mg),Re-run,MeOH Color,pH optimal for assay,Hair Type,Hair Length (cm),Color,Comments,"Worked on sample, but not able to process",Hair Took Extra Time,Repeats
0,1,43518,160804-1,160804-1,17.19,22.990111,N,N,normal,"course, short",3.81,black scalp and tip ends with tan middle,0,0,0,0
1,2,43518,161215 T-1,161215 T-1,25.28,29.120000,N,N,normal,course with split ends,5.02,black with tan tips and scalp end,0,0,0,0
2,3,43518,161215 T-2,161215 T-2,15.85,27.255521,N,N,normal,course with split ends,5.88,black with brown tips,0,0,0,0
3,4,43518,161130-1 O-210 collar,161130-1 O-210 collar,22.84,17.120000,N,N,normal,course with slightly split ends,7.02,black with brown tips,0,0,0,0


## Prep boar cortisol data for merging with other dataframes on Sample ID

In [69]:
# grabbing Sample IDs column in new dataframe, preserving index
prep_boar_cortisol_df = boar_cortisol_df[['Sample ID', 'Sample Number (electronic sheet)', 'Cortisol (pg/mg)']].copy()
prep_boar_cortisol_df['Sample ID'] = prep_boar_cortisol_df['Sample ID'].astype('str')

# removing dashes/spaces & terms 'GPS' + 'collar' from sample IDs
prep_boar_cortisol_df['Sample ID'] = prep_boar_cortisol_df['Sample ID'].apply(lambda row: f_tm.remove_dashes_space_sampleIDs(row))
prep_boar_cortisol_df['Sample ID'] = prep_boar_cortisol_df['Sample ID'].astype('str')

## Saving prepped cortisol data for later retrieval

In [41]:
prep_boar_cortisol_df.to_csv('../excel data/cleaned/prep_boar_cortisol_df.csv', index=False)

In [79]:
prep_boar_cortisol_df

,Sample ID,Sample Number (electronic sheet),Cortisol (pg/mg)
0,1608041,160804-1,22.990111
1,161215T1,161215 T-1,29.120000
2,161215T2,161215 T-2,27.255521
3,1611301O210,161130-1 O-210 collar,17.120000
4,1608081,160803-1,5.635528
5,1607261,160726-1,6.736842
6,1611305O214,161130-5 O-214 collar,4.240000
7,1607262,160726-2,7.087576
8,161216O247,161216 O-247,5.360000
9,161206O231,161206 O-231,29.600000


# Kelly's Data (teloFISH, dicentrics)
---

## Extracting Kelly's boar telomere FISH data

In [38]:
boar_teloFISH_list = f_tm.extract_boar_teloFISH_as_list('../excel data/raw/')

Handling Telomeres - Fukushima Hyb 4 complete.xlsx...


NameError: name 'ssample_data' is not defined

## Formatting telomere FISH data

In [ ]:
# making a dataframe from the list
kelly_boar_teloFISH_df = pd.DataFrame(boar_teloFISH_list, columns=['Sample ID', 'teloFISH data', 'teloFISH means'])

# turning the teloFISH data, which is a series in each row containing 4800 individial telomere lengths, 
# into a list in each row so that it's saved & loaded properly
kelly_boar_teloFISH_df['teloFISH data'] = kelly_boar_teloFISH_df['teloFISH data'].apply(lambda row: row.tolist())
kelly_boar_teloFISH_df['Sample ID'] = kelly_boar_teloFISH_df['Sample ID'].apply(lambda row: f_tm.remove_dashes_space_sampleIDs(row))

## Saving kelly boar teloFISH dataframe for later retrieval

In [ ]:
kelly_boar_teloFISH_df.to_csv('../excel data/cleaned/kelly_boar_teloFISH_df.csv', index=False)

## Loading Kelly boar dicentric data

In [ ]:
kelly_boar_dicentrics = pd.read_excel('../excel data/raw/Fukushima Dicentrics.xlsx')

In [ ]:
kelly_boar_dicentrics.rename(columns={'Pig ID': 'Sample ID'}, inplace=True)
kelly_boar_dicentrics['Sample ID'] = kelly_boar_dicentrics['Sample ID'].apply(lambda row: f_tm.remove_dashes_space_sampleIDs(row))
kelly_boar_dicentrics['Average Dicentrics per cell'] = kelly_boar_dicentrics['Dicentrics']/kelly_boar_dicentrics['Total Scored']                                       
kelly_boar_dicentrics.drop(['Dicentric Images', 'Notes', 'Dicentrics', 'Total Scored'], axis=1, inplace=True)

kelly_boar_dicentrics.head(4)

## Saving Kelly boar dicentrics dataframe for later retrieval

In [ ]:
kelly_boar_dicentrics.to_csv('../excel data/cleaned/kelly_boar_dicentrics_df.csv', index=False)

## Loading new dose exposure data for Kelly's boar 

In [ ]:
kelly_boar_teloFISH_df = pd.read_csv('../excel data/cleaned/kelly_boar_teloFISH_df.csv')

In [ ]:
kelly_boar_dose = pd.read_excel('../excel data/raw/Kelly& Aryn boar_1Dec2018-edited-TH-17Nov19 (2).xlsx',
                               sheet_name=0, skiprows=7, nrows=40)
kelly_boar_dose.head(3)

In [ ]:
kelly_boar_dose['Age (months)'] = kelly_boar_dose['Age (hours)'].apply(lambda x: int(x / (24 * 30)))
kelly_boar_dose['Sample ID'] = kelly_boar_dose['Sample ID'].apply(lambda row: f_tm.remove_dashes_space_sampleIDs(row))
kelly_boar_dose.drop(['QA/AC', 'Unnamed: 3', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', ' '], axis=1, inplace=True)

## Saving Kelly's boar dose exposure data for later retrieval

In [ ]:
kelly_boar_dose.to_csv('../excel data/cleaned/kelly_boar_dose.csv', index=False)

## Briefly looking at sharing samples between kelly's boar w/ teloFISH data vs. the exposure data & cortisol data

In [ ]:
df_list = [trim_kelly_new_exposure_df, prep_boar_cortisol_df]

for df in df_list:
    f_tm.count_shared_sample_IDs(kelly_boar_teloFISH_df, df, print_names='no')

In [ ]:
kelly_teloFISH_IDs = list(kelly_boar_teloFISH_df['Sample ID'].unique())
kelly_exposure_IDs = list(trim_kelly_new_exposure_df['Sample ID'].unique())

a = kelly_teloFISH_IDs
b = kelly_exposure_IDs

mismatched = [sample_ID for sample_ID in a if sample_ID not in b]
print(mismatched)

## Pulling data Tom H request week of 4/27/20

In [17]:
one = aryn_boar_dose_raw[['Sample ID']].copy()
one.shape

two = boar_cortisol_df_raw[['Sample ID']].copy()
two.rename({'Sample Number (electronic sheet)':'Sample ID'}, axis=1, inplace=True)

three = pd.read_csv('../excel data/cleaned/kelly_boar_teloFISH_df.csv')
three = three[['Sample ID']].copy()

In [18]:
cat = pd.concat([one, two, three], axis=0)

def replace_row(row):
    row = row.replace('-', '')
    row = row.replace(' ', '')
    return row

cat = cat.dropna()
cat['Sample ID'] = cat['Sample ID'].apply(lambda row: replace_row(row))
    
cat = (cat
 .dropna()
 .drop_duplicates()
 .reset_index(drop=True))

In [43]:
# two

In [78]:
cat.to_excel('kelly_aryn_cortisol_ids_fukushima_boar_ids_minimal_cleaning.xlsx')

## Combining size data from Tom w/ cortisol data

In [75]:
tom_size_data = pd.read_excel("../excel data/cleaned/Jared's_ids_minimal_cleaning-th-da-10may20_READY.xlsx", 
                              skiprows=list(range(0,6)), usecols=list(range(0,10)))
tom_size_data.drop(['Unnamed: 0', 'Entry', 'Sex', 'Age (wk)'], axis=1, inplace=True)
tom_size_data = tom_size_data.dropna(axis=0, how='any').copy()
tom_size_data.reset_index(drop=True, inplace=True)
tom_size_data['Sample ID'] = tom_size_data['Sample ID'].apply(lambda row: f_tm.remove_dashes_space_sampleIDs(row))
tom_size_data.head(4)

,Sample ID,Month,Year,mass (kg),length (cm),girth (cm)
0,1608081,August,2016,20,68.5,50.5
1,1608013,August,2016,48,117.5,81.8
2,1608031,August,2016,77.2,68.5,50.5
3,1608012,August,2016,56.4,121.2,85.5


In [76]:
total_dose_cortisol = pd.read_csv('../excel data/cleaned/total_dose_cortisol.csv')
merge_tom_data_cortisol = total_dose_cortisol.merge(tom_size_data, on=['Sample ID'])
merge_tom_data_cortisol.drop(3, inplace=True)
merge_tom_data_cortisol.reset_index(drop=True, inplace=True)
merge_tom_data_cortisol.to_csv('../excel data/cleaned/merge_tom_data_cortisol.csv', index=False)

In [78]:
merge_tom_data_cortisol

,Sample ID,Sex,Age (months),"INTERNAL Life time dose, mGy 134+137","EXTERNAL Dose Reasonable Life Time, mGy (134+137)","Reasonable Total Life Time Dose mGy (Int+Ext,134+137)",Dose rate (uGy per h),Cortisol (pg/mg),encoded sex,Month,Year,mass (kg),length (cm),girth (cm)
0,1607261,Female,20,29.200071,584.6400,613.840071,15.849122,6.736842,1,July,2016,38.7,107.0,68.8
1,1607262,Male,20,4.997361,204.6240,209.621361,3.316593,7.087576,0,July,2016,69,128.5,93.6
2,1607264,Female,18,31.137820,88.7040,119.841820,5.162829,5.360000,1,July,2016,43.6,112.9,75.5
3,1611305O214,Female,20,18.404285,584.6400,603.044285,4.167048,4.240000,1,November,2016,55.1,113.0,91.4
4,161206O231,Female,14,0.000000,416.6400,416.640000,14.000000,29.600000,1,December,2016,39.2,111.1,77.8
5,161208O236,Female,11,9.786261,112.8960,122.682261,8.167048,8.320000,1,December,2016,48.6,105.5,82.8
6,161215T1,Female,20,0.005152,1.4616,1.466752,0.100400,29.120000,1,December,2016,43.4,120.2,77.7
7,161215T1,Female,20,0.005152,1.4616,1.466752,0.100400,30.253737,1,December,2016,43.4,120.2,77.7
8,161215T2,Male,14,0.002510,1.0416,1.044110,0.100277,27.255521,0,December,2016,30.3,98.5,66.8
9,161215T2,Male,14,0.002510,1.0416,1.044110,0.100277,23.796954,0,December,2016,30.3,98.5,66.8
